# 필요한 패키지 설치
pip install -r requirements.txt

# Model, Tokenizer 생성

In [2]:
import torch
print(torch.cuda.is_available())  # True여야 GPU 사용 가능
print(torch.cuda.mem_get_info())

True
(5368709120, 6441926656)


In [1]:
# 양자화 매개변수 정의
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4", # 4비트 양자화
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "allganize/Llama-3-Alpha-Ko-8B-Evo"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=2048)
# llm = HuggingFacePipeline(pipeline=pipe)

# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, temperature=0.9, top_p=0.95, repetition_penalty=1.2, do_sample=True)

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

# 아기돼지 삼형제 프롬프트
### 줄거리 요약
옛날 옛적 어느 들판에 어미 돼지와 세 마리 아기 돼지가 살고 있었습니다.
세 돼지가 성장하자 어미 돼지는 이들에게 독립해 각자 집을 짓고 살아가라고 조언했습니다.
첫째 돼지는 게으르고 일하기 싫어 쉽게 지을 수 있는 집을 찾다 결국 주변에서 구한 지푸라기로 집을 빠르게 완성합니다.
그는 스스로에게 만족하며 남은 시간에는 휴식을 취하며 여유롭게 지냈습니다. 둘째 돼지는 첫째보다는 조금 부지런했지만 여전히 편리함을 중시했습니다.
그는 나무를 주워다 조금 더 튼튼한 나무 집을 지었지만, 집 짓는 데 큰 시간은 들이지 않았습니다. 셋째 돼지는 근면하고 신중했습니다.
그는 튼튼한 벽돌을 하나하나 모아 집을 짓기 시작했고, 벽돌을 쌓고 시멘트로 단단히 고정하며 문과 창문, 굴뚝까지 꼼꼼하게 만들었습니다.
집 짓는 데 시간이 오래 걸렸지만, 셋째 돼지는 자신의 집이 가장 안전하다고 믿었습니다. 어느 날, 숲속에서 굶주린 늑대가 나타났습니다.
그는 먹을 것을 찾아 돌아다니다 첫째 돼지의 지푸라기 집을 발견했습니다.
늑대는 문을 두드리며 “아기 돼지야, 문을 열어라. 아니면 숨을 불어 집을 날려버릴 테다!”라고 협박했습니다.
첫째 돼지는 문을 열지 않았고, 늑대는 큰 숨을 들이마신 후 한 번에 지푸라기 집을 날려버렸습니다.
첫째 돼지는 깜짝 놀라 도망쳐 둘째 돼지의 나무 집으로 달려갔습니다. 늑대는 다시 나타나 나무 집 문 앞에서 같은 협박을 했습니다.
두 돼지는 문을 걸어 잠그고 숨었지만, 늑대는 다시 한 번 큰 숨을 들이마셔 나무 집을 무너뜨렸습니다. 둘은 도망쳐 셋째 돼지의 벽돌집으로 향했습니다.
늑대는 이번에는 벽돌집 앞에 서서 세 번째 협박을 했습니다. 하지만 아무리 숨을 불어도 튼튼하게 지어진 벽돌집은 흔들리지 않았습니다.
늑대는 화가 나 굴뚝으로 침입하려 했지만, 셋째 돼지는 미리 벽난로에 물을 끓이고 있었고, 늑대는 뜨거운 물에 데어 소리를 지르며 도망쳤습니다.
그날 이후 세 돼지는 함께 셋째 돼지의 벽돌집에서 살아가며 근면함과 준비의 중요함을 되새기며 늑대의 위험으로부터 안전하게 지냈습니다.

In [21]:
pig_prompt = """
다음은 어린이 투자 교육을 위한 '동화 아기돼지 삼형제 기반 투자 시뮬레이션 게임'입니다.
이 게임은 '아기돼지 삼형제' 동화를 배경으로 하며, 지푸라기 집, 나무 집, 벽돌 집이라는 상징적인 요소를 통해 아이들에게 투자 개념을 쉽고 재미있게 알려주는 것을 목표로 합니다.
당신은 이 게임의 시나리오 디자이너입니다. 아래의 투자종목과 시나리오 작성 지침을 참고하여 7턴 분량의 게임 진행 JSON 데이터를 생성해주세요.
시나리오를 풍성하고 창의력있게 작성하세요.

### 투자 종목(고정)
1. 지푸라기 집
   - risk_level: 고위험 고수익
2. 나무 집
   - risk_level: 균형형
3. 벽돌 집
   - risk_level: 장기 안정형
고위험 고수익에 해당하는 지푸라기 집은 가치의 변동 폭이 크고 뉴스의 영향도 가장 많이 받습니다.
균형형에 해당하는 나무 집은 가치의 변동 폭이 안정적인 편입니다.
장기 안정형에 해당하는 벽돌 집은 가치의 변동 폭이 거의 없지만 뉴스의 영향을 받는 경우가 가장 적습니다.

### 시나리오 작성 지침:
각 턴은 다음 필드를 포함해야 합니다:
- `turn_number`: (int) 현재 턴 번호 (1~7)
- `result`: (str) 턴이 결과 설명
    - turn_number가 1인 턴에서는 이 게임의 목적과 배경 설명을 간단하게 알려주세요.
    - turn_number가 2 이상인 턴에서는 이전 턴의 `news`에서 예보된 사건이 실제로 일어났다는 설명을 넣어주세요.
- `news`: (str) 아기돼지 삼형제 줄거리를 참고하여 일어날 수 있는 사건들을 창의력있게 생성해 어떤 일이 미래에 일어날거라고 설명
- `news_hint`: (str) news에서 일어날거라고 한 일에 대해서 간접적으로 종목들 중 어떤 것을 선택하는 것이 좋을지 추측할 수 있을 만한 힌트
- `stocks`: 각 투자 종목의 상태를 리스트로 표현
    - 'name': (str) 종목 이름 (지푸라기 집 / 나무 집 / 벽돌 집)
    - 'initial_value': (int) 각 집들의 초기 가치 (고정)
    - 'risk_level': (str) 각 종목들이 어떤 투자 유형에 해당하는지 (위에서 지정한 그대로 출력)
    - 'current_value': (int) 해당 턴 종료 후 가치 (뉴스와 사건 반영, 이전 턴의 가치 기준으로 변동)
    - 'description': (str) news_hint에서 어떤 것을 선택하는 것이 좋을지 간접적으로 한 설명에 대해 각 집들이 그 설명과 어떤 식으로 비슷하거나 다른지에 대한 설명

※ 이 게임은 5세에서 10세 사이의 아이들을 위한 것이므로, 어려운 경제 용어나 주식 용어는 사용하지 말고,
  동화 속 이야기나 캐릭터의 행동, 감정 등을 활용해 아이들이 이해하기 쉬운 방식으로 표현해주세요.
  예를 들어, 숫자나 투자 용어 대신 동화 속 장면이나 사건으로 결과를 나타내 주세요.
※
※ 모든 출력은 한국어로 해주세요.
※ 전체 출력은 JSON 리스트 형식으로, 설명 없이 순수 데이터만 출력해 주세요.
"""

# 챗봇 성능(출력) 확인

## test prompt

In [55]:
test_prompt = """
당신은 '동화 기반 어린이 투자 게임'의 시나리오 디자이너입니다.

게임은 '아기돼지 삼형제' 이야기를 바탕으로 하며, 지푸라기 집, 나무 집, 벽돌 집 3가지 투자 종목이 있습니다. 각 종목은 다음과 같은 투자 특성을 가집니다:

1. 지푸라기 집 - 고위험 고수익
2. 나무 집 - 균형형
3. 벽돌 집 - 장기 안정형

이제 1턴 분량의 시나리오 데이터를 JSON 형식으로 만들어주세요.

**다음 형식을 참고하되, 내용은 완전히 새롭게 생성해 주세요.**  
**출력은 JSON 한 덩어리만 하며, 다른 텍스트는 포함하지 마세요.**

(참고 형식)
```json
[
  {
    "turn_number": 1,
    "result": "아이들이 투자 게임에 참여해요.",
    "news": "내일은 바람이 많이 불 거예요.",
    "news_hint": "튼튼한 집이 유리할 수도 있어요.",
    "stocks": [
      {
        "name": "지푸라기 집",
        "initial_value": 200,
        "risk_level": "고위험 고수익",
        "current_value": 190,
        "expectation": "바람에 약해서 가치가 떨어졌어요."
      },
      {
        "name": "나무 집",
        "initial_value": 300,
        "risk_level": "균형형",
        "current_value": 300,
        "expectation": "적당히 튼튼해서 영향이 적었어요."
      },
      {
        "name": "벽돌 집",
        "initial_value": 500,
        "risk_level": "장기 안정형",
        "current_value": 510,
        "expectation": "바람에 강해서 가치가 올랐어요."
      }
    ]
  }
]
"""

In [59]:
test = """
당신은 '아기돼지 삼형제' 이야기를 기반으로 한 어린이 투자 게임의 시나리오 디자이너입니다.

이 게임에는 세 가지 투자 종목이 있습니다:
1. 지푸라기 집 - 고위험 고수익
2. 나무 집 - 균형형
3. 벽돌 집 - 장기 안정형

이제 1턴 분량의 시나리오를 생성하세요. 출력은 JSON 형식으로 하며, 다음 항목을 포함해야 합니다:

- 'turn_number': (int) 턴 번호
- 'result': (str) 어제 일어났던 사건
- 'news': (str) 미래에 일어날 사건
- 'news_hint': (str) 미래 사건에 대해 어떤 종목이 영향을 받을지에 대한 힌트
- 'stocks': (list) 세 가지 집 각각에 대한 정보
  - 'name': (str) 집 이름
  - 'initial_value': (int) 초기 가치
  - 'risk_level': (str) 투자 위험 수준
  - 'current_value': (int) 현재 가치
  - 'expectation': (str) 기대 설명

※ 출력은 JSON 한 덩어리만 하며, 다른 텍스트는 포함하지 마세요.
※ 모든 출력은 한국어로 해 주세요.
※ 형식을 꼭 지켜 주세요.
"""

output = pipe(test)[0]["generated_text"]
print(output)


당신은 '아기돼지 삼형제' 이야기를 기반으로 한 어린이 투자 게임의 시나리오 디자이너입니다.

이 게임에는 세 가지 투자 종목이 있습니다:
1. 지푸라기 집 - 고위험 고수익
2. 나무 집 - 균형형
3. 벽돌 집 - 장기 안정형

이제 1턴 분량의 시나리오를 생성하세요. 출력은 JSON 형식으로 하며, 다음 항목을 포함해야 합니다:

- 'turn_number': (int) 턴 번호
- 'result': (str) 어제 일어났던 사건
- 'news': (str) 미래에 일어날 사건
- 'news_hint': (str) 미래 사건에 대해 어떤 종목이 영향을 받을지에 대한 힌트
- 'stocks': (list) 세 가지 집 각각에 대한 정보
  - 'name': (str) 집 이름
  - 'initial_value': (int) 초기 가치
  - 'risk_level': (str) 투자 위험 수준
  - 'current_value': (int) 현재 가치
  - 'expectation': (str) 기대 설명

※ 출력은 JSON 한 덩어리만 하며, 다른 텍스트는 포함하지 마세요.
※ 모든 출력은 한국어로 해 주세요.
※ 형식을 꼭 지켜 주세요.



## test_with_tiny_llama

In [53]:
import re
import json
import os

# 챗봇 응답 함수
def chat_with_tiny_llama(save_path="data/test.json"):
    user_input = """
당신은 '아기돼지 삼형제' 이야기를 기반으로 한 어린이 투자 게임의 시나리오 디자이너입니다.

이 게임에는 세 가지 투자 종목이 있습니다:
1. 지푸라기 집 - 고위험 고수익
2. 나무 집 - 균형형
3. 벽돌 집 - 장기 안정형

이제 1턴 분량의 시나리오를 생성하세요. 출력은 JSON 형식으로 하며, 다음 항목을 포함해야 합니다:

- 'turn_number': (int) 턴 번호
- 'result': (str) 어제 일어났던 사건
- 'news': (str) 미래에 일어날 사건
- 'news_hint': (str) 미래 사건에 대해 어떤 종목이 영향을 받을지에 대한 힌트
- 'stocks': (list) 세 가지 집 각각에 대한 정보
  - 'name': (str) 집 이름
  - 'initial_value': (int) 초기 가치
  - 'risk_level': (str) 투자 위험 수준
  - 'current_value': (int) 현재 가치
  - 'expectation': (str) 기대 설명

※ 출력은 JSON 한 덩어리만 하며, 다른 텍스트는 포함하지 마세요.
※ 모든 출력은 한국어로 해 주세요.
※ 형식을 꼭 지켜 주세요.
"""

    # system_prompt를 제거하고 바로 user_input만 사용
    input = tokenizer(user_input, return_tensors="pt")
    input_ids = input["input_ids"].to(model.device)
    attention_mask = input["attention_mask"].to(model.device)

    output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.9,
        top_p=0.95,
        repetition_penalty=1.2
    )

    response = tokenizer.decode(output[0], skip_special_tokens=True, errors="ignore")
    print(response)

    # 디렉터리 생성 및 응답 저장
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    with open(save_path, "w", encoding="utf-8") as f:
        f.write(response)
    print(f"✅ 전체 응답 저장 완료: {save_path}")

    return response

In [54]:
chat_with_tiny_llama()


당신은 '아기돼지 삼형제' 이야기를 기반으로 한 어린이 투자 게임의 시나리오 디자이너입니다.

이 게임에는 세 가지 투자 종목이 있습니다:
1. 지푸라기 집 - 고위험 고수익
2. 나무 집 - 균형형
3. 벽돌 집 - 장기 안정형

이제 1턴 분량의 시나리오를 생성하세요. 출력은 JSON 형식으로 하며, 다음 항목을 포함해야 합니다:

- 'turn_number': (int) 턴 번호
- 'result': (str) 어제 일어났던 사건
- 'news': (str) 미래에 일어날 사건
- 'news_hint': (str) 미래 사건에 대해 어떤 종목이 영향을 받을지에 대한 힌트
- 'stocks': (list) 세 가지 집 각각에 대한 정보
  - 'name': (str) 집 이름
  - 'initial_value': (int) 초기 가치
  - 'risk_level': (str) 투자 위험 수준
  - 'current_value': (int) 현재 가치
  - 'expectation': (str) 기대 설명

※ 출력은 JSON 한 덩어리만 하며, 다른 텍스트는 포함하지 마세요.
※ 모든 출력은 한국어로 해 주세요.
※ 형식을 꼭 지켜 주세요.

✅ 전체 응답 저장 완료: data/test.json


"\n당신은 '아기돼지 삼형제' 이야기를 기반으로 한 어린이 투자 게임의 시나리오 디자이너입니다.\n\n이 게임에는 세 가지 투자 종목이 있습니다:\n1. 지푸라기 집 - 고위험 고수익\n2. 나무 집 - 균형형\n3. 벽돌 집 - 장기 안정형\n\n이제 1턴 분량의 시나리오를 생성하세요. 출력은 JSON 형식으로 하며, 다음 항목을 포함해야 합니다:\n\n- 'turn_number': (int) 턴 번호\n- 'result': (str) 어제 일어났던 사건\n- 'news': (str) 미래에 일어날 사건\n- 'news_hint': (str) 미래 사건에 대해 어떤 종목이 영향을 받을지에 대한 힌트\n- 'stocks': (list) 세 가지 집 각각에 대한 정보\n  - 'name': (str) 집 이름\n  - 'initial_value': (int) 초기 가치\n  - 'risk_level': (str) 투자 위험 수준\n  - 'current_value': (int) 현재 가치\n  - 'expectation': (str) 기대 설명\n\n※ 출력은 JSON 한 덩어리만 하며, 다른 텍스트는 포함하지 마세요.\n※ 모든 출력은 한국어로 해 주세요.\n※ 형식을 꼭 지켜 주세요.\n"

# 챗봇 응답 함수 (chat_with_llama)

## 한턴씩 차례로 생성

In [ ]:
import json
import os

def generate_turns_with_context(title: str, system_prompt: str, save_path="data/output_one.json"):
    all_turns = []

    for turn_number in range(1, 8):
        if turn_number == 1:
            user_input = f"""
            {title}를 배경으로 턴 {turn_number}에 대한 투자 시뮬레이션 게임 데이터를 JSON 형식으로 만들어줘.
            ※ 설명 없이 순수 JSON만 출력해 주세요.
            """
        else:
            prev_turn = all_turns[-1]
            prev_news = prev_turn["news"]
            prev_prices = {stock["name"]: stock["current_value"] for stock in prev_turn["stocks"]}
            price_hint = "\\n".join([f"- {k}: {v}" for k, v in prev_prices.items()])

            user_input = f"""
            {title}를 배경으로 턴 {turn_number}에 대한 투자 시뮬레이션 게임 데이터를 JSON 형식으로 만들어줘.

            이전 턴의 뉴스: "{prev_news}"
            이전 턴의 종목별 가격:
            {price_hint}

            ※ 위 내용을 참고하여 다음 턴에서 어떤 일이 일어났는지, 종목들의 가치 변화가 어떻게 나타나는지 반영해 주세요.
            ※ 출력은 순수 JSON 하나만 출력해 주세요.
            """

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ]

        input_ids = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
        if tokenizer.pad_token_id is None:
            tokenizer.pad_token_id = tokenizer.eos_token_id
        attention_mask = input_ids != tokenizer.pad_token_id

        output = model.generate(
            input_ids=input_ids.to(model.device),
            attention_mask=attention_mask.to(model.device),
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            max_new_tokens=1024,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )

        response = tokenizer.decode(output[0], skip_special_tokens=True)

        try:
            turn_data = json.loads(response)
            all_turns.append(turn_data)
        except json.JSONDecodeError:
            print(f"❌ 턴 {turn_number}에서 JSON 파싱 실패\n{response}")
            break

    # 디렉토리 생성 및 응답 저장
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(all_turns, f, ensure_ascii=False, indent=2)

    print(f"✅ 전체 {len(all_turns)}턴 생성 완료: {save_path}")
    return all_turns

## test prompt

In [ ]:
test_prompt = """
### Instruction:
당신은 어린이를 위한 투자 시뮬레이션 게임의 시나리오 작가입니다.

이 게임은 '작은 투자 마법사'가 되어 3개의 상점(빵집, 서커스단, 마법연구소)에 마법 코인을 투자하며 3턴 동안 진행됩니다.

아래 형식처럼, 아이가 쉽게 이해할 수 있는 투자 시뮬레이션 게임의 **3턴 분량 JSON 시나리오를 생성하세요.**

각 턴마다 다음 형식의 JSON을 출력하세요 (설명 없이 JSON만 출력):

[
  {
    "turn_number": 1,
    "result": "당신은 마법 마을의 어린 투자 마법사가 되어 세 상점에 마법 코인을 투자해 마을을 도와야 해요!",
    "news": "내일은 비가 많이 올 예정이에요!",
    "news_hint": "비가 오면 야외 공연이 어려워질지도 몰라요.",
    "stocks": [
      {
        "name": "빵집",
        "initial_value": 30,
        "risk_level": "저위험",
        "current_value": 30,
        "expectation": "비가 와도 사람들은 빵을 사러 와요."
      },
      {
        "name": "서커스단",
        "initial_value": 40,
        "risk_level": "중위험",
        "current_value": 35,
        "expectation": "비가 오면 공연장이 젖을 수 있어요."
      },
      {
        "name": "마법연구소",
        "initial_value": 50,
        "risk_level": "고위험",
        "current_value": 52,
        "expectation": "비가 오면 연구에 집중할 수 있어요."
      }
    ]
  },
  ...
]

❗주의사항:
- 총 3턴 분량을 위와 같은 형식으로 출력해주세요.
- 각 턴은 `news` → `result` 순으로 인과관계가 이어지도록 작성해주세요.
- 아동이 이해할 수 있도록 간단한 말, 동화 같은 표현을 써 주세요.
- 출력은 반드시 **JSON 형식의 배열**만 작성하고, 설명이나 여는 말 없이 출력해주세요.

### Response:
"""


## 7턴을 한번에 생성

In [22]:
import re
import json
import os

# 챗봇 응답 함수
def chat_with_llama(system_prompt, save_path="data/output.json"):
#     user_input = """
# 작은 투자 마법사 기반 3턴짜리 게임 시나리오를 작성해줘.
# """


    # messages = [
    #     {"role": "system", "content": system_prompt},
    #     {"role": "user", "content": user_input}
    # ]

    prompt = f"""### Instruction:
{system_prompt.strip()}

### Response:
"""

    # input = tokenizer.apply_chat_template(
    #     messages,
    #     tokenize=True,
    #     add_generation_prompt=True,
    #     return_tensors="pt"
    # )

    # inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(model.device)
    attention_mask = inputs["attention_mask"].to(model.device)


    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id

    # attention_mask = input != tokenizer.pad_token_id

    output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask.to(model.device),
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=2048,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )
    
    # response = tokenizer.decode(output[0], skip_special_tokens=True, errors="replace")
    response = tokenizer.decode(output[0], skip_special_tokens=True, errors="ignore")
    print(response)

    # 디렉터리 생성 및 응답 저장
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    with open(save_path, "w", encoding="utf-8") as f:
        f.write(response)
    print(f"✅ 전체 응답 저장 완료: {save_path}")

    return response

In [23]:
chat_with_llama(test_prompt)


### Instruction:
### Instruction:
당신은 어린이를 위한 투자 시뮬레이션 게임의 시나리오 작가입니다.

이 게임은 '작은 투자 마법사'가 되어 3개의 상점(빵집, 서커스단, 마법연구소)에 마법 코인을 투자하며 3턴 동안 진행됩니다.

각 턴마다 다음 형식의 JSON을 출력하세요 (설명 없이 JSON만 출력):

[
  {
    "turn_number": 1,
    "result": "당신은 마법 마을의 어린 투자 마법사가 되어 세 상점에 마법 코인을 투자해 마을을 도와야 해요!",
    "news": "내일은 비가 많이 올 예정이에요!",
    "news_hint": "비가 오면 야외 공연이 어려워질지도 몰라요.",
    "stocks": [
      {
        "name": "빵집",
        "initial_value": 30,
        "risk_level": "저위험",
        "current_value": 30,
        "expectation": "비가 와도 사람들은 빵을 사러 와요."
      },
      {
        "name": "서커스단",
        "initial_value": 40,
        "risk_level": "중위험",
        "current_value": 35,
        "expectation": "비가 오면 공연장이 젖을 수 있어요."
      },
      {
        "name": "마법연구소",
        "initial_value": 50,
        "risk_level": "고위험",
        "current_value": 52,
        "expectation": "비가 오면 연구에 집중할 수 있어요."
      }
    ]
  },
 ...
]

❗주의사항:
- 총 3턴 분량을 위와 같은 형식으로 출력해주세요.
- 각 턴은 `news` → `result` 순으로 인과관계가 이어지도록 

'### Instruction:\n### Instruction:\n당신은 어린이를 위한 투자 시뮬레이션 게임의 시나리오 작가입니다.\n\n이 게임은 \'작은 투자 마법사\'가 되어 3개의 상점(빵집, 서커스단, 마법연구소)에 마법 코인을 투자하며 3턴 동안 진행됩니다.\n\n각 턴마다 다음 형식의 JSON을 출력하세요 (설명 없이 JSON만 출력):\n\n[\n  {\n    "turn_number": 1,\n    "result": "당신은 마법 마을의 어린 투자 마법사가 되어 세 상점에 마법 코인을 투자해 마을을 도와야 해요!",\n    "news": "내일은 비가 많이 올 예정이에요!",\n    "news_hint": "비가 오면 야외 공연이 어려워질지도 몰라요.",\n    "stocks": [\n      {\n        "name": "빵집",\n        "initial_value": 30,\n        "risk_level": "저위험",\n        "current_value": 30,\n        "expectation": "비가 와도 사람들은 빵을 사러 와요."\n      },\n      {\n        "name": "서커스단",\n        "initial_value": 40,\n        "risk_level": "중위험",\n        "current_value": 35,\n        "expectation": "비가 오면 공연장이 젖을 수 있어요."\n      },\n      {\n        "name": "마법연구소",\n        "initial_value": 50,\n        "risk_level": "고위험",\n        "current_value": 52,\n        "expectation": "비가 오면 연구에 집중할 수 있어요."\n      }\n    ]\n  },\n ...\n]\n\n❗주의사항:\n- 총 3턴 분량을 위와 같은 형식으로 출력해주